In [1]:
import pandas as pd
import numpy as np
from pandas.io import sql
import requests
from bs4 import BeautifulSoup
import psycopg2
import re

# Read data

In [4]:
from sqlalchemy import create_engine
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to connect to the northwind database
database = 'climbing'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
connection = create_engine(conn_str)

In [6]:
df_climbs_sql = pd.read_sql('select * from climbs',connection,index_col='index')
df_climbs_sql['logs'] = [eval(flat) for flat in df_climbs_sql['logs_flat']]
df_climbs_sql = df_climbs_sql.drop('logs_flat',axis=1)
df_climbs_sql

,id,url,climbname,rating,grade,crag_id,num_comments,type,num_worded_comments,logs
index,,,,,,,,,,
3166,3166,https://www.ukclimbing.com/logbook/c.php?i=3166,Millwood's Wall,0,S 4a,4,0,trad,0,{}
3167,3167,https://www.ukclimbing.com/logbook/c.php?i=3167,Vanishing Point,0,E1 5b,4,2,trad,0,"{'187067': '', '63530': ''}"
3168,3168,https://www.ukclimbing.com/logbook/c.php?i=3168,The Thing,3,E2 5c,4,124,trad,24,"{'253245': '', '206609': 'Joe led this in one ..."
3169,3169,https://www.ukclimbing.com/logbook/c.php?i=3169,Rootorooni,1,E3 5c,4,0,trad,0,{}
3170,3170,https://www.ukclimbing.com/logbook/c.php?i=3170,Cobweb Crack,2,VS 5a,4,181,trad,41,"{'185136': '', '197782': '', '6106': '', '2410..."
...,...,...,...,...,...,...,...,...,...,...
560510,560510,https://www.ukclimbing.com/logbook/c.php?i=560510,El chivo loco,0,6b,27432,0,lead,0,{}
560511,560511,https://www.ukclimbing.com/logbook/c.php?i=560511,Victorking,0,6b+,27432,0,lead,0,{}
560512,560512,https://www.ukclimbing.com/logbook/c.php?i=560512,Puto Maldonado,0,6c+,27432,0,lead,0,{}


In [7]:
df_climbs_sql.type.value_counts()

lead       209346
boulder    110784
trad       102743
other       70109
Name: type, dtype: int64

In [8]:
df_crags_sql = pd.read_sql('select * from crags',connection,index_col='index')
df_crags_sql['climb_list'] = [eval(flat) for flat in df_crags_sql['climbs_flat']]
df_crags_sql = df_crags_sql.drop('climbs_flat',axis=1)
df_crags_sql

df_crags_sql

,url,cragname,climbs,rocktype,altitude,faces,lat,long,id,climb_list
index,,,,,,,,,,
4,https://www.ukclimbing.com/logbook/crag.php?id=4,Dinas Cromlech,79,Rhyolite,500.0,S,53.0917,-4.0489,4,"[(3166, 0), (3167, 0), (3168, 3), (3169, 1), (..."
5,https://www.ukclimbing.com/logbook/crag.php?id=5,Bus Stop Quarry,115,Slate,450.0,None,53.1294,-4.1058,5,"[(141016, 1), (136481, 1), (134391, 1), (1309,..."
6,https://www.ukclimbing.com/logbook/crag.php?id=6,Craig Pant Ifan (Tremadog),176,Dolerite,NaN,S,52.9436,-4.1311,6,"[(152703, 0), (100182, 1), (2561, 2), (2562, 2..."
7,https://www.ukclimbing.com/logbook/crag.php?id=7,St. Govan's Head,151,Limestone,12.0,SW,51.5977,-4.9277,7,"[(401447, 0), (138693, 1), (58187, 1), (150671..."
8,https://www.ukclimbing.com/logbook/crag.php?id=8,Carreg-y-Barcud Area,128,Sandstone (hard),13.0,S,51.8719,-5.2350,8,"[(503566, 0), (503567, 0), (503568, 0), (50356..."
...,...,...,...,...,...,...,...,...,...,...
27429,https://www.ukclimbing.com/logbook/crag.php?id...,Gap Bon ami boulders,0,UNKNOWN,NaN,None,48.8067,-64.2198,27429,[]
27430,https://www.ukclimbing.com/logbook/crag.php?id...,Metal Expert,8,UNKNOWN,NaN,None,45.5454,25.5181,27430,"[(560519, 3), (560520, 3), (560521, 0), (56052..."
27431,https://www.ukclimbing.com/logbook/crag.php?id...,Las Palomas,3,UNKNOWN,NaN,None,28.1159,-16.5589,27431,"[(560451, 2), (560452, 2), (560453, 2)]"


In [9]:
df_users_sql = pd.read_sql('select * from users',connection,index_col='index')
df_users_sql['climbs'] = [eval(flat) for flat in df_users_sql['climbs_flat']]
df_users_sql = df_users_sql.drop('climbs_flat',axis=1)
df_users_sql

,id,logbook_url,profile_url,username,num climbs,climbs
index,,,,,,
100002,100002,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,Stephanelericolais,3,"[42012, 147314, 18853]"
10002,10002,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,Piers Harley,100,"[4348, 4360, 38698, 9981, 9961, 9987, 9968, 99..."
100025,100025,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,george5699,16,"[33002, 34313, 31435, 38715, 33342, 86139, 352..."
100031,100031,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,Tak,17,"[10969, 10969, 10941, 10971, 10917, 10342, 103..."
100045,100045,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,smeaton,46,"[9636, 9631, 9630, 70266, 9595, 9594, 9593, 96..."
...,...,...,...,...,...,...
99962,99962,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,spilla,19,"[14076, 14075, 14074, 14073, 14071, 14070, 140..."
99967,99967,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,Andrewbirkett,70,"[81721, 51757, 37451, 33152, 33154, 86212, 954..."
99978,99978,https://www.ukclimbing.com/logbook/showlog.php...,https://www.ukclimbing.com/user/profile.php?id...,Jamie Nobbs,48,"[8603, 8602, 34950, 8596, 32990, 32275, 47459,..."


In [10]:
df_users_sql[df_users_sql['username']=='maxmurmann'].climbs.tolist()

[['250574',
  '246663',
  '408053',
  '191165',
  '28936',
  '30210',
  '30209',
  '257747',
  '30358',
  '42675']]

In [11]:
climb_list = df_users_sql[df_users_sql['username']=='maxmurmann'].climbs.tolist()[0]

In [12]:
climb_list = [int(climb) for climb in climb_list]

In [13]:
df_climbs_sql.loc[climb_list]

,id,url,climbname,rating,grade,crag_id,num_comments,type,num_worded_comments,logs
index,,,,,,,,,,
250574,250574,https://www.ukclimbing.com/logbook/c.php?i=250574,Happiness is Coming,0,5.11d,18857,3,lead,0,"{'279250': '', '31674': '', '56428': ''}"
246663,246663,https://www.ukclimbing.com/logbook/c.php?i=246663,Tortuga,2,5.12a,18857,3,lead,1,"{'279250': '', '105453': 'Very bouldery crux.'..."
408053,408053,https://www.ukclimbing.com/logbook/c.php?i=408053,Trey Lizardos,1,5.11d,18857,1,lead,1,{'279250': 'Great first 11d/7a for a boulderer...
191165,191165,https://www.ukclimbing.com/logbook/c.php?i=191165,Director of Humor Affairs,2,5.11a,18857,7,lead,1,{'279250': 'Classic route with some great heel...
28936,28936,https://www.ukclimbing.com/logbook/c.php?i=28936,Oedipus Direct,1,f6B+,22,163,boulder,30,"{'192528': 'After the original, jumped off at ..."
30210,30210,https://www.ukclimbing.com/logbook/c.php?i=30210,Kraken,1,6b+,75,662,lead,82,"{'279330': '', '174913': '', '235676': '', '16..."
30209,30209,https://www.ukclimbing.com/logbook/c.php?i=30209,Brean Dream,1,6a+,75,663,lead,82,"{'279330': 'Sketchy first move, rest of it is ..."
257747,257747,https://www.ukclimbing.com/logbook/c.php?i=257747,Do you get Vouchers with This?,0,5c,75,251,lead,28,"{'174913': '', '235676': '', '213950': '', '12..."
30358,30358,https://www.ukclimbing.com/logbook/c.php?i=30358,La Marie Rose (Red 22),3,f6A,1652,670,boulder,157,"{'264238': '', '156192': '', '266168': '3rd tr..."


# Start of NLP

In [ ]:
df_crags_short = df_crags_sql.loc[df_climbs_sql.crag_id.unique()]

In [ ]:
df_crags_short

In [ ]:
def comments_flat(com_dic):
    flat = ""
    for comment in com_dic:
        flat += com_dic[comment] + ' '
    return flat

In [ ]:
df_climbs_sql['comments_flat'] = df_climbs_sql['logs'].map(comments_flat)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
comment_mat = cv.fit_transform(df_climbs_sql['comments_flat'])

In [ ]:
comment_df = pd.DataFrame(comment_mat.asformat('array'),columns=cv.get_feature_names(),index=df_climbs_sql.index)

In [2]:
wiki = "https://en.wikipedia.org/wiki/Glossary_of_climbing_terms"

r = requests.get(wiki)
soup = BeautifulSoup(r.text,'html.parser')
body = soup.find('div',attrs={'class':'mw-parser-output'})
list_of_words = [word.text.replace('\n','') for word in body.find_all('p')[3:]]

In [4]:
list_of_words

flat = ""
for word in list_of_words:
    flat += word.lower() + ' '
list_of_words = re.findall('\w+',flat)

In [8]:
list_of_words

['abalakov',
 'thread',
 'ablation',
 'zone',
 'abseil',
 'acr',
 'alpine',
 'cock',
 'ring',
 'add',
 'on',
 'adze',
 'aid',
 'climbing',
 'aider',
 'alpine',
 'climbing',
 'alpine',
 'knee',
 'alpine',
 'start',
 'american',
 'death',
 'triangle',
 'anchor',
 'approach',
 'arête',
 'arm',
 'bar',
 'arqué',
 'ascend',
 'ascender',
 'aspect',
 'atc',
 'automatic',
 'belay',
 'b',
 'grade',
 'bachar',
 'ladder',
 'back',
 'clipping',
 'back',
 'step',
 '1',
 '8',
 '9',
 'bail',
 'ball',
 'nut',
 'barn',
 'door',
 'bashie',
 'belay',
 'belay',
 'device',
 'belay',
 'loop',
 'belay',
 'off',
 'belay',
 'on',
 'belay',
 'station',
 'benightment',
 'bergschrund',
 'or',
 'schrund',
 'beta',
 'beta',
 'flash',
 'bicycle',
 'bidoigt',
 'big',
 'wall',
 'biner',
 'bivy',
 'or',
 'bivvy',
 'bivy',
 'bag',
 'bollard',
 'bolt',
 'bolt',
 'chopping',
 'bomb',
 'proof',
 'anchor',
 'book',
 'or',
 'open',
 'book',
 'booty',
 'bosun',
 's',
 'chair',
 'bouldering',
 'bridging',
 'bucket',
 'bumbly',

In [9]:
import csv
with open('list_of_words.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(list_of_words)